In [ ]:
import numpy as np

# st-HOSVD

In [ ]:
def choose_rank(s, energy_tol):
    tail_energy = np.cumsum(s[::-1]**2)[::-1]
    k = np.searchsorted(tail_energy < energy_tol, True)
    return max(1, k), tail_energy[k] if k < len(tail_energy) else 0.0


In [ ]:
def st_hosvd_seq(tensor, rel_tol=1e-8, ranks=None):

    core = tensor.copy()
    abs_tol = (rel_tol * np.linalg.norm(core))**2
    factors, dims = [], list(core.shape)

    for mode in range(core.ndim):
        unfolded = np.reshape(np.moveaxis(core, mode, 0), (dims[mode], -1))
        U, s, Vh = np.linalg.svd(unfolded, full_matrices=False)

        if ranks is None:
            k, lost = choose_rank(s, abs_tol / (core.ndim - mode))
            abs_tol -= lost
        else:
            k = ranks[mode]

        factors.append(U[:, :k])
        core = (s[:k, None] * Vh[:k]).reshape([k] + dims[:mode] + dims[mode+1:])
        dims[mode] = k
        core = np.moveaxis(core, 0, mode)

    return core, factors, dims

In [ ]:

sizes = (10, 20, 30, 40)
i, j, k, l = np.ogrid[:sizes[0], :sizes[1], :sizes[2], :sizes[3]]
T = 1.0 / (i + j + k + l + 1)

# %%
core, factors, ranks = st_hosvd_seq(T, rel_tol=1e-10)
print("Ranks:", ranks)

# %%
def tucker_to_tensor(core, factors):
    tensor = core
    for mode, U in enumerate(factors):
        tensor = np.tensordot(U, tensor, axes=(1, mode)) 
        tensor = np.moveaxis(tensor, 0, mode)           
    return tensor

approx = tucker_to_tensor(core, factors)
rel_err = np.linalg.norm(approx - T) / np.linalg.norm(T)
print("Relative reconstruction error:", rel_err)


Ranks: [10, 11, 12, 12]
Relative reconstruction error: 3.561131561135256e-11
